In [66]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager

In [67]:
def get_last_news():
    url_base = "https://redplanetscience.com/"
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url_base)
    html = browser.html

    soup = bs(html, "html.parser")
    news = soup.find('div',id='news')
    last = news.find("div", class_ = 'list_text')
    title = last.find("div", class_ = 'content_title').text 
    para = last.find("div", class_ = 'article_teaser_body').text 
    list_date = last.find("div", class_ = 'list_date').text 
    
    browser.quit() 

    return list_date, title, para

In [68]:
list_date,title,para = get_last_news()     
print(f"title = {title}")
print(f"text = {para}")
print(f"date = {list_date}")



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\alans\.wdm\drivers\chromedriver\win32\92.0.4515.43\chromedriver.exe] found in cache


title = 10.9 Million Names Now Aboard NASA's Perseverance Mars Rover
text = As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.
date = July 27, 2021


In [73]:
def get_mars_image():
    url_base = "https://spaceimages-mars.com/"
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url_base)
    html = browser.html
    soup = bs(html, "html.parser")
    header=soup.find('img', class_="headerimage")
    url = url_base + header['src']
    browser.quit()
    return url




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\alans\.wdm\drivers\chromedriver\win32\92.0.4515.43\chromedriver.exe] found in cache


In [88]:

url = get_mars_image() 
print(f"Mars image = {url}")

'https://spaceimages-mars.com/image/featured/mars2.jpg'